# Day 11
## Puzzle 1

In [87]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [88]:
input_file = 'input_1.txt'
# input_file = 'test_input_1.txt'
# input_file = 'test_input_2.txt'

Read input line.

In [89]:
with open(file=input_file, mode="r") as file:
    line = file.read().strip().split()

Iterate (blink) 25 times and update the line according to the three rules. The answer is the length of the line in the last iteration.

In [90]:
puzzle_1_line = line.copy()

n = 25

for i in range(n):
    new_line = []

    for val in puzzle_1_line:
        if int(val) == 0:
            new_line.append('1')

        elif len(val) % 2 == 0:
            new_line.append(str(int(val[:int(len(val)/2)])))
            new_line.append(str(int(val[int(len(val)/2):])))

        else:
            new_line.append(str(int(val)*2024))

    puzzle_1_line = new_line


In [91]:
len(new_line)

194482

## Puzzle 2

Oooops... Iterating 75 times, using the same strategy as in part one, is not really optimal...

Instead, we iterate 75 times (at most) and connect edges between nodes (the stone numbers) according to the rules. This way, we don't calculate the same thing more than once.

In [92]:
puzzle_2_line = line.copy()
start_nodes = []
edges = []
n = 75

for i in range(n):
    new_line = []

    for val in puzzle_2_line:
        if val not in start_nodes:
            start_nodes.append(val)

        else:
            continue

        if int(val) == 0:
            new_val = '1'
            edges.append((val, new_val))

            if new_val not in start_nodes:
                new_line.append(new_val)

        elif len(val) % 2 == 0:
            new_val_1 = str(int(val[:int(len(val)/2)]))
            new_val_2 = str(int(val[int(len(val)/2):]))
            edges.append((val, new_val_1))
            edges.append((val, new_val_2))

            if new_val_1 not in start_nodes:
                new_line.append(new_val_1)

            if new_val_2 not in start_nodes:
                new_line.append(new_val_2)

        else:
            new_val = str(int(val)*2024)
            edges.append((val, new_val))

            if new_val not in start_nodes:
                new_line.append(new_val)

    if not new_line:
        break
    
    puzzle_2_line = new_line

In [93]:
start_nodes[:10]

['0', '27', '5409930', '828979', '4471', '3', '68524', '170', '1', '2']

In [94]:
edges[:10]

[('0', '1'),
 ('27', '2'),
 ('27', '7'),
 ('5409930', '10949698320'),
 ('828979', '828'),
 ('828979', '979'),
 ('4471', '44'),
 ('4471', '71'),
 ('3', '6072'),
 ('68524', '138692576')]

Create a multigraph from the edges.

In [95]:
D = nx.MultiDiGraph()
_ = D.add_edges_from(edges)  # Catch the annoying output with the variable _.

Plot the graph (only for test input).

In [96]:
if input_file.startswith('test'):
    plt.figure(figsize=(20, 20))
    nx.draw_circular(D, with_labels=True)

Extract the adjacency matrix of the graph.

In [97]:
A = nx.linalg.adjacency_matrix(D).toarray()

Instead of raising the adjacency matrix to the 75:th power (which takes forever for the puzzle input), only multiply the rows corresponding to the starting values with 75 times. The sum of the resulting matrix elements is the answer we are looking for. The values of the martix correspond to the number of walks of length 75 from the starting nodes to the other nodes of the graph.

In [98]:
current_rows = [node in line for node in D.nodes]

for i in tqdm(range(n)):
    current_rows = current_rows@A

100%|██████████| 75/75 [00:14<00:00,  5.14it/s]


In [99]:
int(sum(current_rows))

232454623677743